# Hard Disks Failure Prediction

### Reading the CSV file

In [0]:
from pyspark.sql import SparkSession
import requests
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
# Create a Spark session
spark = SparkSession.builder.appName("Hard Disks Failure Prediction") \
    .config("spark.databricks.delta.formatCheck.enabled", "false") \
    .getOrCreate()
# #spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")
# # Replace with the raw URL of your desired CSV file
url = "https://raw.githubusercontent.com/PacktPublishing/Apache-Spark-for-Machine-Learning/main/Chapter05/Data/disk-smart.csv"
df = spark.read.csv(url, header=True, inferSchema=True)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1170325858090373>:11
      8 # #spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")
      9 # # Replace with the raw URL of your desired CSV file
     10 url = "https://raw.githubusercontent.com/PacktPublishing/Apache-Spark-for-Machine-Learning/main/Chapter05/Data/disk-smart.csv"
---> 11 df = spark.read.csv(url, header=True, inferSchema=True)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:729, in DataFrameReader.csv(self, path, s

In [0]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml.clustering import LDA
from pyspark.ml.evaluation import ClusteringEvaluator
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_20newsgroups

# Initialize SparkSession
spark = SparkSession.builder.appName("LDA_Clustering_20Newsgroups").getOrCreate()

# Load the 20-Newsgroups dataset
newsgroups = fetch_20newsgroups(subset="all", remove=("headers", "footers", "quotes"))
data = newsgroups.data

# Create a DataFrame with a single column "text"
df = spark.createDataFrame([(text.split(),) for text in data], ["text"])

# Tokenize and vectorize the text data
cv = CountVectorizer(inputCol="text", outputCol="raw_features", vocabSize=10000)
cv_model = cv.fit(df)
df_vectorized = cv_model.transform(df)

In [0]:
df.show(5)

+--------------------+
|                text|
+--------------------+
|[I, am, sure, som...|
|[My, brother, is,...|
|[Finally, you, sa...|
|[Think!, It's, th...|
|[1), I, have, an,...|
+--------------------+
only showing top 5 rows



In [0]:
# Apply IDF to the vectorized features
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(df_vectorized)
df_final = idf_model.transform(df_vectorized)

In [0]:
# Perform LDA clustering
lda = LDA(k=20, maxIter=10, featuresCol="features")
model = lda.fit(df_final)
transformed_data = model.transform(df_final)
transformed_data.show(5)

+--------------------+--------------------+--------------------+--------------------+
|                text|        raw_features|            features|   topicDistribution|
+--------------------+--------------------+--------------------+--------------------+
|[I, am, sure, som...|(10000,[0,1,2,3,4...|(10000,[0,1,2,3,4...|[0.13752336400707...|
|[My, brother, is,...|(10000,[0,3,5,6,8...|(10000,[0,3,5,6,8...|[3.09095381139583...|
|[Finally, you, sa...|(10000,[0,1,2,3,4...|(10000,[0,1,2,3,4...|[9.89246123665615...|
|[Think!, It's, th...|(10000,[0,1,2,3,4...|(10000,[0,1,2,3,4...|[1.44948481848543...|
|[1), I, have, an,...|(10000,[0,1,2,3,5...|(10000,[0,1,2,3,5...|[2.19989041161527...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [0]:
# Evaluate clustering using Silhouette score
evaluator = ClusteringEvaluator()
silhouette_score = evaluator.evaluate(transformed_data)
print(f"Silhouette Score: {silhouette_score:.4f}")

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-2908121104079526>:3
      1 # Evaluate clustering using Silhouette score
      2 evaluator = ClusteringEvaluator()
----> 3 silhouette_score = evaluator.evaluate(transformed_data)
      4 print(f"Silhouette Score: {silhouette_score:.4f}")

File /databricks/spark/python/pyspark/ml/evaluation.py:111, in Evaluator.evaluate(self, dataset, params)
    109         return self.copy(params)._evaluate(dataset)
    110     else:
--> 111         return self._evaluate(dataset)
    112 else:
    113     raise TypeError("Params must be a param map but got %s." % type(params))

File /databricks/spark/python/pyspark/ml/evaluation.py:148, in JavaEvaluator._evaluate(self, dataset)
    146 self._transfer_params_to_java()
    147 assert self._java_obj is not None
--> 148 return self._java_obj.evaluate(dataset._jdf)

File /databricks/spark/pyth

In [0]:
# Visualize the clusters (using the first two components)
topics = model.describeTopics(10)
topic_words = topics.select("termIndices").rdd.flatMap(lambda x: x[0]).collect()
vocab = cv_model.vocabulary
topic_words_list = [[vocab[i] for i in [indices]] for indices in topic_words]

plt.figure(figsize=(10, 6))
for i, words in enumerate(topic_words_list):
    plt.scatter(transformed_data.select("topicDistribution").collect()[i][0][0],
                transformed_data.select("topicDistribution").collect()[i][0][1],
                label=f"Topic {i}", s=100)
    for word in words:
        plt.annotate(word, (transformed_data.select("topicDistribution").collect()[i][0][0],
                            transformed_data.select("topicDistribution").collect()[i][0][1]),
                     textcoords="offset points", xytext=(0, 5), ha="center")

plt.title("LDA Clustering (20-Newsgroups)")
plt.xlabel("Topic 0")
plt.ylabel("Topic 1")
plt.legend()
plt.show()